# Module 05: Resolving Merge Conflicts

**Difficulty**: ⭐⭐ Intermediate

**Estimated Time**: 60-90 minutes

**Prerequisites**: 
- [Module 00: Setup and Introduction](00_setup_and_introduction.ipynb)
- [Module 01: Git Fundamentals](01_git_fundamentals.ipynb)
- [Module 02: Branching and Merging](02_branching_and_merging.ipynb)
- [Module 03: Remote Repositories and GitHub](03_remote_repositories_and_github.ipynb)
- [Module 04: Collaborating with Pull Requests](04_collaborating_with_pull_requests.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand what causes merge conflicts and why they occur
2. Recognize and interpret conflict markers in files
3. Resolve conflicts manually using a text editor
4. Use merge tools to resolve conflicts more efficiently
5. Prevent conflicts through good collaboration practices
6. Handle complex conflict scenarios confidently
7. Apply best practices for conflict resolution

---

## 1. What Are Merge Conflicts?

### The Concept

A **merge conflict** occurs when Git can't automatically determine which changes should take precedence.

```
Main Branch                Feature Branch
    |
    A
    |
    B ────────┐            B
    |         │            |
    C (file.txt line 1:    D (file.txt line 1:
       "Hello World")         "Greetings Earth")
    |         │            |
    ↓         └──merge────→ CONFLICT!
```

Git says: "Both branches modified the same line. You decide!"

### Why Conflicts Happen

Conflicts occur when:
1. **Same line edited**: Two branches modify the same line differently
2. **File deleted vs modified**: One branch deletes a file, another modifies it
3. **File renamed**: One branch renames a file, another modifies original
4. **Different line endings**: Windows (CRLF) vs Unix (LF) conflicts

### When Conflicts Are Detected

- During `git merge`
- During `git rebase`
- During `git pull` (which includes merge)
- During `git cherry-pick`

---

## 2. Creating a Conflict (For Learning)

Let's intentionally create a conflict to practice resolving it.

### Setup Repository

In [ ]:
import os

# Create practice directory
practice_dir = "../outputs/conflict_practice"
os.makedirs(practice_dir, exist_ok=True)

print(f"Created directory: {practice_dir}")

In [ ]:
# Initialize repository
%cd {practice_dir}
!git init

# Create initial file
with open("story.txt", "w") as f:
    f.write("""Chapter 1: The Beginning

Once upon a time, there was a developer.
They loved to code every day.
The end.
""")

!git add story.txt
!git commit -m "Initial commit: Add story"
print("Initial commit created!")

### Create Conflicting Changes

We'll modify the same line on two different branches.

In [ ]:
# Modify on main branch
!git switch main

with open("story.txt", "w") as f:
    f.write("""Chapter 1: The Beginning

Once upon a time, there was a brilliant developer.
They loved to code every day.
The end.
""")

!git commit -am "Describe developer as brilliant"
print("Main branch modified!")

In [ ]:
# Go back to commit before the change
!git switch -c feature/alternative-story HEAD~1

# Make different change to same line
with open("story.txt", "w") as f:
    f.write("""Chapter 1: The Beginning

Once upon a time, there was a passionate developer.
They loved to code every day.
The end.
""")

!git commit -am "Describe developer as passionate"
print("Feature branch modified!")

### Trigger the Conflict

In [ ]:
# Try to merge feature into main
!git switch main
!git merge feature/alternative-story

**Notice the output**:
```
Auto-merging story.txt
CONFLICT (content): Merge conflict in story.txt
Automatic merge failed; fix conflicts and then commit the result.
```

---

## 3. Understanding Conflict Markers

### Anatomy of a Conflict

Git marks conflicts in your files like this:

```
<<<<<<< HEAD
Once upon a time, there was a brilliant developer.
=======
Once upon a time, there was a passionate developer.
>>>>>>> feature/alternative-story
```

### Breaking It Down

**`<<<<<<< HEAD`**
- Marks the start of the conflict
- `HEAD` = current branch (main) version

**Content between `<<<<<<<` and `=======`**
- Your current branch's changes

**`=======`**
- Separator between the two versions

**Content between `=======` and `>>>>>>>`**
- Incoming branch's changes

**`>>>>>>> feature/alternative-story`**
- Marks the end of the conflict
- Shows the branch being merged

### View the Conflict

In [ ]:
# View conflicted file
!cat story.txt

In [ ]:
# Check status
!git status

**Status shows**:
```
Unmerged paths:
  both modified:   story.txt
```

---

## 4. Resolving Conflicts Manually

### Step 1: Open the Conflicted File

You have three options:

**Option 1: Keep HEAD version (current branch)**
```
Once upon a time, there was a brilliant developer.
```

**Option 2: Keep incoming version (feature branch)**
```
Once upon a time, there was a passionate developer.
```

**Option 3: Combine both (best solution here)**
```
Once upon a time, there was a brilliant and passionate developer.
```

**Option 4: Write something completely new**
```
Once upon a time, there was an extraordinary developer.
```

### Step 2: Edit the File

In [ ]:
# Resolve conflict by combining both changes
with open("story.txt", "w") as f:
    f.write("""Chapter 1: The Beginning

Once upon a time, there was a brilliant and passionate developer.
They loved to code every day.
The end.
""")

print("Conflict resolved! Markers removed, content combined.")

### Step 3: Mark as Resolved

In [ ]:
# Stage the resolved file
!git add story.txt

# Check status
!git status

### Step 4: Complete the Merge

In [ ]:
# Commit the merge
!git commit -m "Merge feature/alternative-story: Combine character descriptions"

# View history
!git log --oneline --graph --all

---

## 5. Using Git Commands to Resolve Conflicts

### Accept One Side Completely

Sometimes you want to take one version entirely.

In [ ]:
# Create another conflict scenario
!git switch main
with open("config.txt", "w") as f:
    f.write("setting=production")
!git add config.txt
!git commit -m "Set production config"

!git switch -c feature/dev-config HEAD~1
with open("config.txt", "w") as f:
    f.write("setting=development")
!git add config.txt
!git commit -m "Set development config"

# Try to merge
!git switch main
!git merge feature/dev-config || echo "Conflict occurred!"

### Accept Current Branch Version (ours)

In [ ]:
# Keep our version (main branch)
!git checkout --ours config.txt
!git add config.txt

# Or for all files:
# !git checkout --ours .

!cat config.txt

### Accept Incoming Branch Version (theirs)

In [ ]:
# Keep their version (feature branch)
!git checkout --theirs config.txt
!git add config.txt

# Or for all files:
# !git checkout --theirs .

!cat config.txt

In [ ]:
# Complete merge
!git commit -m "Merge dev-config: Accept development settings"

### Abort a Merge

In [ ]:
# If you want to cancel the merge and start over
# !git merge --abort

# This returns you to the state before the merge attempt
print("Use 'git merge --abort' to cancel a merge in progress")

---

## 6. Using Merge Tools

### Built-in Git Mergetool

Git supports visual merge tools that make conflict resolution easier.

**Popular merge tools**:
- **VS Code**: Built-in merge conflict resolver
- **Meld**: Visual diff and merge tool
- **KDiff3**: Cross-platform merge tool
- **P4Merge**: Perforce visual merge tool
- **Beyond Compare**: Professional diff tool

### Configure VS Code as Merge Tool

In [ ]:
# Set VS Code as default merge tool
!git config --global merge.tool vscode
!git config --global mergetool.vscode.cmd 'code --wait $MERGED'

print("VS Code configured as merge tool!")

### Using the Merge Tool

In [ ]:
# When you have a conflict, run:
# !git mergetool

# This opens the merge tool for each conflicted file
# After resolving in the tool, save and close
# Git automatically stages the resolved file

print("Run 'git mergetool' to launch visual merge tool")

### VS Code Conflict Resolution Features

When you open a conflicted file in VS Code, you see:

```
<<<<<<< HEAD (Current Change)
Once upon a time, there was a brilliant developer.
=======
Once upon a time, there was a passionate developer.
>>>>>>> feature/alternative-story (Incoming Change)
```

**VS Code provides buttons**:
- **Accept Current Change**: Keep HEAD version
- **Accept Incoming Change**: Keep feature branch version
- **Accept Both Changes**: Include both versions
- **Compare Changes**: Side-by-side comparison

---

## 7. Complex Conflict Scenarios

### Multiple Files in Conflict

In [ ]:
# View all conflicted files
!git diff --name-only --diff-filter=U

# Or use status
!git status | grep "both modified"

**Strategy**: Resolve files one at a time

```bash
# Resolve first file
git add file1.txt

# Resolve second file
git add file2.txt

# When all resolved
git commit
```

### Binary File Conflicts

Binary files (images, PDFs) can't be merged automatically.

In [ ]:
# For binary files, choose one version:

# Keep current version
# !git checkout --ours image.png

# Keep incoming version
# !git checkout --theirs image.png

# Then stage it
# !git add image.png

print("Binary files: choose one version with --ours or --theirs")

### Deleted vs Modified Conflicts

One branch deletes a file, another modifies it.

In [ ]:
# If you want to delete the file
# !git rm filename.txt

# If you want to keep the modified version
# !git add filename.txt

print("Deleted vs modified: explicitly add or rm the file")

---

## 8. Preventing Merge Conflicts

### Best Practices

1. **Pull frequently**: Stay in sync with main branch
```bash
git switch main
git pull
git switch feature-branch
git merge main  # or: git rebase main
```

2. **Make small, focused commits**: Easier to resolve conflicts

3. **Communicate with team**: Let others know what files you're working on

4. **Use feature branches**: Keep work isolated

5. **Don't edit the same files simultaneously**: Coordinate with team

6. **Use .gitattributes for merge strategies**:
```
# .gitattributes
*.json merge=union  # Combine changes
*.md merge=union    # Combine markdown changes
```

### File Organization

**Good**: Modular, separate files
```
src/
  user.py
  payment.py
  shipping.py
```
Each developer works on different files.

**Bad**: Everything in one file
```
src/
  main.py  # 5000 lines!
```
Everyone edits the same file → more conflicts.

### Communication Patterns

**Use GitHub Issues/Projects**:
```
Issue #42: "Refactoring authentication module"
Assignee: Alice
Status: In Progress

→ Bob knows not to edit auth files
```

**Regular sync meetings**:
- Daily standups
- Weekly planning
- Discuss who's working on what

---

## 9. Exercises

### Exercise 1: Basic Conflict Resolution

1. Create a repository with a file
2. Create two branches
3. Modify the same line differently on each branch
4. Merge and resolve the conflict manually
5. Verify the merge was successful

In [ ]:
# Your code here for Exercise 1


### Exercise 2: Multiple File Conflicts

1. Create a repository with 3 files
2. Create a feature branch
3. Modify all 3 files on main
4. Modify all 3 files on feature branch (same lines)
5. Merge and resolve all conflicts
6. Practice using both manual and `--ours`/`--theirs` approaches

In [ ]:
# Your code here for Exercise 2


### Exercise 3: Practice Aborting

1. Create a conflict scenario
2. Start resolving it
3. Decide you want to start over
4. Use `git merge --abort`
5. Try again with a different approach

In [ ]:
# Your code here for Exercise 3


---

## 10. Conflict Resolution Checklist

### When a Conflict Occurs

1. **Don't panic!** Conflicts are normal and fixable

2. **Check status**: `git status` shows conflicted files

3. **Understand the conflict**: Look at both versions

4. **Choose resolution strategy**:
   - Manual edit (most common)
   - Accept ours (`--ours`)
   - Accept theirs (`--theirs`)
   - Use merge tool

5. **Resolve each file**: Remove conflict markers

6. **Test the code**: Make sure it works!

7. **Stage resolved files**: `git add <file>`

8. **Complete merge**: `git commit`

9. **Push if needed**: `git push`

### Common Mistakes to Avoid

❌ **Leaving conflict markers**: Code won't compile!
```python
<<<<<<< HEAD
def function():
=======
```

❌ **Not testing after resolution**: Merged code might not work

❌ **Accepting wrong version**: Understand what each side does

❌ **Force pushing over conflicts**: You'll lose changes

---

## 11. Knowledge Check

Before moving on, ensure you can answer:

1. What causes merge conflicts?
2. What do the conflict markers `<<<<<<<`, `=======`, and `>>>>>>>` mean?
3. How do you resolve a conflict manually?
4. What's the difference between `--ours` and `--theirs`?
5. How do you abort a merge?
6. How can you prevent conflicts?
7. What should you do after resolving all conflicts?

### Practical Checklist

Can you:
- [ ] Identify when a merge conflict occurs
- [ ] Read and understand conflict markers
- [ ] Resolve a conflict manually
- [ ] Use `--ours` and `--theirs` to accept one version
- [ ] Abort a merge if needed
- [ ] Complete a merge after resolving conflicts
- [ ] Test code after conflict resolution

---

## 12. Summary

In this module, you learned:

- ✅ What merge conflicts are and why they occur
- ✅ How to read and interpret conflict markers
- ✅ Multiple strategies for resolving conflicts
- ✅ How to use `--ours` and `--theirs` for quick resolution
- ✅ How to set up and use merge tools
- ✅ Best practices for preventing conflicts
- ✅ How to handle complex conflict scenarios

Conflicts are a normal part of collaborative development. With practice, resolving them becomes second nature!

---

## 13. Next Steps

**Next Module**: [Module 06: Git Best Practices](06_git_best_practices.ipynb)

In the next module, you'll learn:
- Commit message conventions
- .gitignore patterns and strategies
- When and how to commit
- Branch naming conventions
- Repository structure best practices
- Security practices (avoiding credential leaks)

Take a break and practice resolving conflicts before continuing!

---

**Excellent work! See you in Module 06!**